<a href="https://colab.research.google.com/github/eyreynational/Assiment-datascient1/blob/main/YOLOV8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
# Pip install method (recommended)
%pip install ultralytics
!pip install ultralytics

In [ ]:
# Pip install method (recommended)
!pip install ultralytics
!pip install squarify # Install squarify package


# Importing the required libraries
from ultralytics import YOLO
import squarify # Now you can import squarify
import matplotlib.pyplot as plt
import cv2
import os
import random
import pandas as pd
import matplotlib.image as mpimg
import seaborn as sns

sns.set_style('darkgrid')

%matplotlib inline

In [ ]:
# Define the paths to the images and labels directories
train_images = "/content/drive/MyDrive/Master/Dataset/YOLO/trian/images"
train_labels = "/content/drive/MyDrive/Master/Dataset/YOLO/trian/labels"

test_images = "/content/drive/MyDrive/Master/Dataset/YOLO/test/images"
test_labels = "/content/drive/MyDrive/Master/Dataset/YOLO/test/labels"


# Get a list of all the image files in the training images directory
image_files = os.listdir(train_images)

# Choose 16 random image files from the list
random_images = random.sample(image_files, 16)

# Set up the plot
fig, axs = plt.subplots(4, 4, figsize=(16, 16))

# Loop over the random images and plot the object detections
for i, image_file in enumerate(random_images):
    row = i // 4
    col = i % 4

    # Load the image
    image_path = os.path.join(train_images, image_file)
    image = cv2.imread(image_path)

    # Load the labels for this image
    label_file = os.path.splitext(image_file)[0] + ".txt"
    label_path = os.path.join(train_labels, label_file)
    with open(label_path, "r") as f:
        labels = f.read().strip().split("\n")

    # Loop over the labels and plot the object detections
    # Loop over the labels and plot the object detections
    for label in labels:
        if len(label.split()) != 5:
            continue
        class_id, x_center, y_center, width, height = map(float, label.split())
        x_min = int((x_center - width/2) * image.shape[1])
        y_min = int((y_center - height/2) * image.shape[0])
        x_max = int((x_center + width/2) * image.shape[1])
        y_max = int((y_center + height/2) * image.shape[0])
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 3)


    # Show the image with the object detections
    axs[row, col].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axs[row, col].axis('off')

plt.show()

In [ ]:
# Load an image using OpenCV
image = cv2.imread("/content/drive/MyDrive/Master/Dataset/YOLO/test/images/Batcterial_spot (281).JPG")

# Get the size of the image
height, width, channels = image.shape
print(f"The image has dimensions {width}x{height} and {channels} channels.")

In [ ]:
pip install wandb


In [ ]:
import wandb
wandb.login(key="53f6a7ea5457ca3231dd5f3c3e6aa8dbe4d5fc72")


In [ ]:
# Loading a pretrained model
model = YOLO('yolov8x.pt')

# Training the model
model.train(data = '/content/drive/MyDrive/Master/Dataset/YOLO/Data.yaml',
            epochs = 70,
            imgsz = height,
            seed = 42,
            batch = 8,
            workers = 4)

In [ ]:
%matplotlib inline
# read in the results.csv file as a pandas dataframe
df = pd.read_csv('/content/runs/detect/train/results.csv')
df.columns = df.columns.str.strip()

# create subplots using seaborn
fig, axs = plt.subplots(nrows=5, ncols=2, figsize=(15, 15))

# plot the columns using seaborn
sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0,0])
sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[0,1])
sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1,0])
sns.lineplot(x='epoch', y='metrics/precision(B)', data=df, ax=axs[1,1])
sns.lineplot(x='epoch', y='metrics/recall(B)', data=df, ax=axs[2,0])
sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2,1])
sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[3,0])
sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[3,1])
sns.lineplot(x='epoch', y='val/cls_loss', data=df, ax=axs[4,0])
sns.lineplot(x='epoch', y='val/dfl_loss', data=df, ax=axs[4,1])

# set titles and axis labels for each subplot
axs[0,0].set(title='Train Box Loss')
axs[0,1].set(title='Train Class Loss')
axs[1,0].set(title='Train DFL Loss')
axs[1,1].set(title='Metrics Precision (B)')
axs[2,0].set(title='Metrics Recall (B)')
axs[2,1].set(title='Metrics mAP50 (B)')
axs[3,0].set(title='Metrics mAP50-95 (B)')
axs[3,1].set(title='Validation Box Loss')
axs[4,0].set(title='Validation Class Loss')
axs[4,1].set(title='Validation DFL Loss')

# add suptitle and subheader
plt.suptitle('Training Metrics and Loss', fontsize=24)

# adjust top margin to make space for suptitle
plt.subplots_adjust(top=0.8)

# adjust spacing between subplots
plt.tight_layout()

plt.show()

In [ ]:
import os
import pandas as pd

# Function to check YOLO label format
def check_label_format(label_path):
    with open(label_path, 'r') as f:
        for line in f:
            values = line.strip().split()
            if len(values) != 5:
                print(f"Incorrect number of values in label: {label_path}")
            try:
                [float(value) for value in values]  # Check if all values are numerical
            except ValueError:
                print(f"Non-numerical value found in label: {label_path}")

# Modify paths to match your dataset location
train_data_path = '/content/drive/MyDrive/Master/Dataset/YOLO/trian'
label_dir = os.path.join(train_data_path, 'labels')  # Example path to labels
label_files = [os.path.join(label_dir, file) for file in os.listdir(label_dir) if file.endswith('.txt')]

# Loop through label files and check formats
for label_file in label_files:
    check_label_format(label_file)

In [ ]:
%matplotlib inline
# Loading the best performing model
model = YOLO('/content/runs/detect/train/weights/best.pt')

# Evaluating the model on the test dataset
# data is explicitly provided
metrics = model.val(data='/content/drive/MyDrive/Master/Dataset/YOLO/Data.yaml', conf=0.25, split='val')




In [ ]:
# !pip install ultralytics
# !pip install seaborn
# import seaborn as sns
# import matplotlib.pyplot as plt
# from ultralytics import YOLO
# import pandas as pd

In [ ]:
%matplotlib inline
# Create the barplot
ax = sns.barplot(x=['mAP50-95', 'mAP50', 'mAP75'], y=[metrics.box.map, metrics.box.map50, metrics.box.map75])

# Set the title and axis labels
ax.set_title('YOLO Evaluation Metrics')
ax.set_xlabel('Metric')
ax.set_ylabel('Value')

# Set the figure size
fig = plt.gcf()
fig.set_size_inches(8, 6)

# Add the values on top of the bars
for p in ax.patches:
    ax.annotate('{:.3f}'.format(p.get_height()), (p.get_x() + p.get_width() / 2, p.get_height()), ha='center', va='bottom')

# Show the plot
plt.show()

In [ ]:


%matplotlib inline
# Reading the confusion matrix image file
img = mpimg.imread('/content/runs/detect/train/confusion_matrix.png')

# Plotting the confusion matrix image
fig, ax = plt.subplots(figsize = (15, 15))

ax.imshow(img)
ax.axis('off');



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:


# Function to perform  detections
def ship_detect(img_path):

    # Read the image
    img = cv2.imread(img_path)

    # Pass the image through the detection model and get the result
    detect_result = model(img)

    # Plot the detections
    detect_img = detect_result[0].plot()

    # Convert the image to RGB format
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    return detect_img



In [ ]:
import os
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Path to your test image directory
custom_image_dir = '/content/drive/MyDrive/Master/Dataset/YOLO/test/images'

# Get the list of image files
image_files = os.listdir(custom_image_dir)

# Select 25 random images
selected_images = random.sample(image_files, 25)

# Create subplots
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(25, 25))

# Define your object detection function
def ship_detect(img_path):
    # Read image
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Run detection using your model
    results = model(img_rgb)  # assuming this is your YOLO model
    boxes = results[0].boxes.data.cpu().numpy()  # [x1, y1, x2, y2, conf, cls]
    class_names = model.names  # class ID to name mapping

    for box in boxes:
        x1, y1, x2, y2, conf, cls_id = box
        label = f"{class_names[int(cls_id)]}: {conf:.4f}"  # 4 decimal places
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # Draw box and label
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 0, 0), 2, cv2.LINE_AA)

    return img_rgb

# Plot detections
for i, img_file in enumerate(selected_images):
    row_idx = i // 5
    col_idx = i % 5

    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()


In [ ]:
import os
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np

# Path to your test image directory
custom_image_dir = '/content/drive/MyDrive/Master/Dataset/YOLO/test/images'

# Get the list of image files
image_files = os.listdir(custom_image_dir)

# Select 25 random images
selected_images = random.sample(image_files, 25)

# Create subplots
fig, axes = plt.subplots(nrows=4, ncols=5, figsize=(25, 25))

# Object detection function with enhanced labels
def ship_detect(img_path):
    # Read and convert image
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # Run detection using your YOLO model
    results = model(img_rgb)
    boxes = results[0].boxes.data.cpu().numpy()  # x1, y1, x2, y2, conf, cls
    class_names = model.names

    for box in boxes:
        x1, y1, x2, y2, conf, cls_id = box
        label_text = f"{class_names[int(cls_id)]}: {conf:.4f}"  # 4 decimals
        x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

        # Draw bounding box
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Calculate label size
        (text_w, text_h), baseline = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)

        # Adjust text position
        text_x = x1
        text_y = y1 - 10 if y1 - 10 > text_h else y1 + text_h + 10

        # Draw background rectangle
        cv2.rectangle(img_rgb, (text_x, text_y - text_h), (text_x + text_w, text_y + baseline), (0, 0, 0), -1)

        # Put label text
        cv2.putText(img_rgb, label_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX,
                    0.5, (0, 256, 256), 1, cv2.LINE_AA)

    return img_rgb

# Plot detection results
for i, img_file in enumerate(selected_images):
    row_idx = i // 5
    col_idx = i % 5

    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust layout spacing
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()


In [ ]:
import random
import os # Import the os module
import matplotlib.pyplot as plt # Import matplotlib.pyplot

# Define the directory where the custom images are stored
custom_image_dir = '/content/drive/MyDrive/Master/Dataset/YOLO/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Select 25 random images from the list
selected_images = random.sample(image_files, 16)

# Create a figure with subplots for each image
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))

# Iterate over the selected images and plot each one
for i, img_file in enumerate(selected_images):

    # Compute the row and column index of the current subplot

    row_idx = i // 4
    col_idx = i % 4

    # Load the current image and run object detection
    img_path = os.path.join(custom_image_dir, img_file)
    detect_img = ship_detect(img_path)

    # Plot the current image on the appropriate subplot
    axes[row_idx, col_idx].imshow(detect_img)
    axes[row_idx, col_idx].axis('off')

# Adjust the spacing between the subplots
plt.subplots_adjust(wspace=0.05, hspace=0.05)
plt.show()

In [ ]:
def ship_detect(img_path):
    # Read the image
    img = cv2.imread(img_path)

    # Run detection
    detect_result = model(img)

    # Plot the detection results (for visual display)
    detect_img = detect_result[0].plot()
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    # Extract confidence scores from results
    scores = detect_result[0].boxes.conf.cpu().numpy()

    # Round to 4 decimal places
    scores = [round(score, 4) for score in scores]

    return detect_img, scores


In [ ]:
import matplotlib.pyplot as plt
import os
import random
import cv2 # Import cv2 as it is used in ship_detect
import numpy as np # Import numpy as it is used in ship_detect

# Define the directory where the custom images are stored
custom_image_dir = '/content/drive/MyDrive/Master/Dataset/YOLO/test/images'

# Get the list of image files in the directory
image_files = os.listdir(custom_image_dir)

# Define the ship_detect function
def ship_detect(img_path):
    # Read the image
    img = cv2.imread(img_path)

    # Run detection
    detect_result = model(img)

    # Plot the detection results (for visual display)
    detect_img = detect_result[0].plot()
    detect_img = cv2.cvtColor(detect_img, cv2.COLOR_BGR2RGB)

    # Extract confidence scores from results
    scores = detect_result[0].boxes.conf.cpu().numpy()

    # Round to 4 decimal places
    scores = [round(score, 4) for score in scores]

    return detect_img, scores


# Randomly select 25 images
selected_images = random.sample(image_files, 25)

# Plotting
fig, axes = plt.subplots(5, 5, figsize=(15, 15))
for i, img_file in enumerate(selected_images):
    row, col = divmod(i, 5)
    img_path = os.path.join(custom_image_dir, img_file)

    # Get detection image and confidence scores
    detect_img, scores = ship_detect(img_path)

    # Display image
    axes[row, col].imshow(detect_img)
    axes[row, col].axis('off')

    # Show the top-1 score (or all if you want)
    if scores:
        axes[row, col].set_title(f"Score: {scores[0]:.4f}")
    else:
        axes[row, col].set_title("No detections")

plt.tight_layout()
plt.show()